In [7]:
import csv
import os

In [8]:
disease_folder = 'autism'
date_time_folder = '2018-10-28-7-42-17'
data_folder =  os.path.expanduser('~/data1/complex_disorders/data/%s/cohorts/%s/' %(disease_folder, date_time_folder))

In [9]:
asd_tags = [7495, 7957, 2449, 5357, 11868, 12523, 7529, 5372, 9023]
adhd_tags = [11532, 2177, 2151]
mm_tags = [2892, 4085, 8642, 3091, 372]
tags = [asd_tags, mm_tags, adhd_tags]

TRIM_LEN = 100

In [10]:
with open(os.path.join(data_folder, 'ordered_ehrs.csv')) as f:
    rd = csv.reader(f)
    ordered_ehrs = []
    for r in rd:
        ordered_ehrs += [list(map(int, r))]
        
with open(os.path.join(data_folder, 'mrn_classes.csv')) as f:
    rd = csv.reader(f)
    mrn_classes = {}
    next(rd)
    for r in rd:
        mrn_classes[r[0]] = r[1::]
        
with open(os.path.join(data_folder, 'ordered_mrns.csv')) as f:
    rd = csv.reader(f)
    list_classes = []
    ordered_mrns = []
    for r in rd:
        ordered_mrns.append(r[0])
        list_classes.append(mrn_classes[str(r[0])][0])        
        
with open('cohort-vocab.csv') as f:
    rd = csv.reader(f)
    vocab = {}
    next(rd)
    for r in rd:
        vocab[r[1]] = r[0]

with open('mt_to_ix.csv') as f:
    rd = csv.reader(f)
    old_vocab = {}
    #next(rd)
    for r in rd:
        old_vocab[r[1]] = int(r[0])

In [11]:
trimmed_ehrs = []
new_ordered_mrn = []
new_list_classes = []
for idx, e in enumerate(ordered_ehrs):
    e_tmp = list(filter(lambda x: x>=0, e))
    if len(e_tmp) > TRIM_LEN:
        c = list_classes[idx][0]
        #print(c)
        #print(tags[int(c)-1])
        #print(e)
        if int(c) > 0:
            tmp = []
            for t in tags[int(c)-1]:
                if t in e_tmp:
                    tmp.append(e_tmp.index(t))
            first_diag_idx = min(tmp)
            ctrl = e_tmp[first_diag_idx:len(e_tmp)]
            if len(ctrl) >= TRIM_LEN:
                new_ordered_mrn.append(ordered_mrns[idx])
                new_list_classes += list_classes[idx]
                trimmed_ehrs.append(e_tmp[first_diag_idx:first_diag_idx+TRIM_LEN])
            else:
                new_ordered_mrn.append(ordered_mrns[idx])
                new_list_classes += list_classes[idx]
                trimmed_ehrs.append([e_tmp[first_diag_idx]]+e_tmp[0:TRIM_LEN-1])
        else: 
            new_ordered_mrn.append(ordered_mrns[idx])
            new_list_classes += list_classes[idx]
            trimmed_ehrs.append(e_tmp[0:TRIM_LEN])

    elif len(e_tmp) == TRIM_LEN:
        new_ordered_mrn.append(ordered_mrns[idx])
        new_list_classes += list_classes[idx]
        trimmed_ehrs.append(e_tmp[0:TRIM_LEN])

In [18]:
new_vocab = {}
count = 0
for trim_l in trimmed_ehrs:
    for tok in trim_l:
        if tok not in new_vocab:
            new_vocab[tok] = count
            count += 1
            
new_trimmed_ehrs = []
for te in trimmed_ehrs:
    tmp = []
    for t in te:
        tmp.append(new_vocab[t])
    new_trimmed_ehrs.append(tmp)

tags_trimmed = [[new_vocab[i] for i in asd_tags if i in new_vocab.keys()], 
                [new_vocab[i] for i in mm_tags if i in new_vocab.keys()],
                [new_vocab[i] for i in adhd_tags if i in new_vocab.keys()]]

print(tags_trimmed)

[[4376, 4309, 506, 1769, 8502, 9801, 3228, 3714, 4628], [1087, 1800, 1712, 0], [7890, 904, 423]]


In [20]:
counts = {'0':0, '1':0, '2':0, '3':0}
for i in new_list_classes:
    counts[str(i)] += 1
print(len(trimmed_ehrs))
print("OTH (no overlapping): {0} -- ASD: {1} -- MM: {2} -- ADHD: {3}".format(counts['0'], counts['1'], counts['2'], counts['3']))

4512
OTH (no overlapping): 1238 -- ASD: 405 -- MM: 1405 -- ADHD: 1464


In [21]:
with open(os.path.join(data_folder, 'trimmed_ehrs.csv'), 'w') as f:
    wr = csv.writer(f, delimiter=',')
    for w in new_trimmed_ehrs:
        wr.writerow(w)

with open(os.path.join(data_folder, 'TRIMMEDordered_mrns.csv'), 'w') as f:
    wr = csv.writer(f, delimiter=',')
    for w in new_ordered_mrn:
        wr.writerow([w])
        
with open(os.path.join(data_folder, 'TRIMMEDcohort-vocab.csv'), 'w') as f:
    wr = csv.writer(f, delimiter=',')
    for old, new in new_vocab.items():
        wr.writerow([vocab[str(old_vocab[str(old)])], new])
        
with open(os.path.join(data_folder, 'TRIMMEDordered_labels.csv'), 'w') as f:
    wr = csv.writer(f, delimiter=',')
    for w in new_list_classes:
        wr.writerow(w)